# **Edat mitja de les edificacions**


BCN Open Data publica aquestes dades actualitzades cada any però només per les seccions censals de la ciutat de Barcelona.

D'altra banda, IDESCAT publica les mateixes dades per totes les seccions censals de Catalunya pero de manera decennal (les últimes dades són del 2021).

Aquest notebook neteja les dades d'ambdues fonts i les integra en un sol dataset d'edat mitjana de les edificacions.


In [ ]:
ROOT = "/content/drive/Shareddrives/GESTIÓ DE PROJECTES/X. Entregable/"

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from pyproj import Proj, transform
import numpy as np
import pandas as pd
import seaborn as sns
from shapely import wkt
import json
import datetime
import csv
import pandas as pd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### **Edat mitja de les edificacions (BCN Open Data)**

Edat mitjana de les edificacions cadastrals a la ciutat de Barcelona.

Les dades publicades corresponen a l'1 de gener de cada any.

* **Font**: BCN Open Data
* **Nom**: Edat mitjana de les edificacions cadastrals de la ciutat de Barcelona
* **URL**: https://opendata-ajuntament.barcelona.cat/data/es/dataset/est-cadastre-edificacions-edat-mitjana
* **Periodicitat**: anual (actualitzat sense retràs)
* **Agregació geogràfica**: secció censal (només del municipi de Barcelona)

De la URL, descarregar tots els CSVs dels anys disponibles i afegir els paths a la cel·la de codi d'abaix.

In [ ]:
EDAT_MITJANA_EDIF_2018_CSV = ROOT + 'cleaning/raw_data/edat_mitjana_edificis/2018_edificacions_edat_mitjana.csv'
EDAT_MITJANA_EDIF_2019_CSV = ROOT + 'cleaning/raw_data/edat_mitjana_edificis/2019_edificacions_edat_mitjana.csv'
EDAT_MITJANA_EDIF_2020_CSV = ROOT + 'cleaning/raw_data/edat_mitjana_edificis/2020_edificacions_edat_mitjana.csv'
EDAT_MITJANA_EDIF_2021_CSV = ROOT + 'cleaning/raw_data/edat_mitjana_edificis/2021_edificacions_edat_mitjana.csv'
EDAT_MITJANA_EDIF_2022_CSV = ROOT + 'cleaning/raw_data/edat_mitjana_edificis/2022_edificacions_edat_mitjana.csv'
EDAT_MITJANA_EDIF_2023_CSV = ROOT + 'cleaning/raw_data/edat_mitjana_edificis/2023_edificacions_edat_mitjana.csv'

In [ ]:
edat_mitjana_2018 = pd.read_csv(EDAT_MITJANA_EDIF_2018_CSV, delimiter=',')
edat_mitjana_2019 = pd.read_csv(EDAT_MITJANA_EDIF_2019_CSV, delimiter=',')
edat_mitjana_2020 = pd.read_csv(EDAT_MITJANA_EDIF_2020_CSV, delimiter=',')
edat_mitjana_2021 = pd.read_csv(EDAT_MITJANA_EDIF_2021_CSV, delimiter=',')
edat_mitjana_2022 = pd.read_csv(EDAT_MITJANA_EDIF_2022_CSV, delimiter=',')
edat_mitjana_2023 = pd.read_csv(EDAT_MITJANA_EDIF_2023_CSV, delimiter=',')
edat_mitjana_2023.head()

,Any,Codi_districte,Nom_districte,Codi_barri,Nom_barri,Seccio_censal,Edat_mitjana
0,2023,1,Ciutat Vella,1,el Raval,1,98.9
1,2023,1,Ciutat Vella,1,el Raval,2,108.9
2,2023,1,Ciutat Vella,1,el Raval,3,109.3
3,2023,1,Ciutat Vella,1,el Raval,4,92.6
4,2023,1,Ciutat Vella,1,el Raval,5,116.6


In [ ]:
concat = [edat_mitjana_2018, edat_mitjana_2019, edat_mitjana_2020, edat_mitjana_2021, edat_mitjana_2022, edat_mitjana_2023]
edat_mitjana = pd.concat(concat, ignore_index=True)

edat_mitjana['Codi_districte'] = edat_mitjana['Codi_districte'].astype(str).str.zfill(2)
edat_mitjana['Codi_barri'] = edat_mitjana['Codi_barri'].astype(str).str.zfill(2)
edat_mitjana['Seccio_censal'] = edat_mitjana['Seccio_censal'].astype(str).str.zfill(3)

INE_CODI_PROVINCIA_BARCELONA = "08"
INE_CODI_MUNICIPI_BARCELONA = "019"
edat_mitjana["CUSEC"] = INE_CODI_PROVINCIA_BARCELONA + INE_CODI_MUNICIPI_BARCELONA + edat_mitjana['Codi_districte'] + edat_mitjana['Seccio_censal']
edat_mitjana = edat_mitjana.drop(columns=["Codi_districte", "Nom_districte", "Codi_barri", "Nom_barri", "Seccio_censal"]).rename(columns={"Any":"any"})

display(edat_mitjana.head())

,any,Edat_mitjana,CUSEC
0,2018,93.8,0801901001
1,2018,105.0,0801901002
2,2018,104.3,0801901003
3,2018,88.5,0801901004
4,2018,111.0,0801901005


### **Edat mitja de les edificacions (IDESCAT)**

El Cens de població i habitatges proporciona el recompte de la població resident al territori amb periodicitat decennal. També proporciona informació sobre les principals característiques demogràfiques i socials de la població i de les llars i famílies. Entre aquestes dades es troba la edat mitja de les edificacions per secció censal.

* **Font**: Generalitat de Catalunya - IDESCAT
* **Nom**: Cens de població i habitatges
* **URL**: https://www.idescat.cat/dades/obertes/censph
* **Periodicitat**: decennal (amb 2 anys de retràs)
* **Agregació geogràfica**: secció censal

De la URL, descarregar el dataset "Indicadors d'habitatges - Seccions censals" del últim any disponible i afegir els paths a la següent cel·la de codi.

Com aquest dataset només té dades d'un any, cal especificar el periode d'estudi per tal de replicar les dades a tots els anys necessaris.

In [ ]:
INDICADORS_HABITATGE_CSV = ROOT + 'cleaning/raw_data/edat_mitjana_edificis/2021_indicadors_habitatge_idescat.csv'

In [ ]:
FROM_YEAR = 2018 # (included)
TO_YEAR = 2022 # (included)

In [ ]:
FULL_MAP_FILE = ROOT + "cleaning/clean_data/full_map.csv"
full_map = pd.read_csv(FULL_MAP_FILE, dtype={"CUSEC":str, "CUMUN":str, "CSEC":str, "CDIS":str, "CMUN":str, "CUDIS":str, "CUBAR":str, "CBAR":str})

In [ ]:
indicadors_habitatge = pd.read_csv(INDICADORS_HABITATGE_CSV, delimiter=';')[["any", "secció censal", "concepte", "valor"]]
indicadors_habitatge["valor"] = indicadors_habitatge["valor"].str.replace(",",".").astype(float)
indicadors_habitatge = indicadors_habitatge.dropna()
indicadors_habitatge = indicadors_habitatge.rename(columns={"secció censal": "CUSEC"})
indicadors_habitatge = indicadors_habitatge[indicadors_habitatge['CUSEC'].str.startswith('08')]
indicadors_habitatge['CUSEC'] = indicadors_habitatge['CUSEC'].str[:5] + indicadors_habitatge['CUSEC'].str[6:]

indicadors_habitatge = indicadors_habitatge.merge(full_map[["CUSEC", "AB_Region"]], on="CUSEC", how="left")
indicadors_habitatge["AB_Region"] = indicadors_habitatge["AB_Region"].replace({np.nan: False})
indicadors_habitatge = indicadors_habitatge[indicadors_habitatge["AB_Region"]].drop(columns=["AB_Region"])
indicadors_habitatge = pd.pivot_table(indicadors_habitatge, values='valor', index=['any', 'CUSEC'], columns=['concepte']).reset_index().rename_axis(None, axis=1)
indicadors_habitatge["Edat_mitjana"] = ((indicadors_habitatge["proporció d'habitatges principals construïts fins al 1950 (%)"]/100) * (TO_YEAR-1900) +
                                        (indicadors_habitatge["proporció d'habitatges principals construïts de 1951 a 1970 (%)"]/100) * (TO_YEAR-1960) +
                                        (indicadors_habitatge["proporció d'habitatges principals construïts de 1971 a 1980 (%)"]/100) * (TO_YEAR-1975) +
                                        (indicadors_habitatge["proporció d'habitatges principals construïts de 1981 a 2000 (%)"]/100) * (TO_YEAR-1990) +
                                        (indicadors_habitatge["proporció d'habitatges principals construïts en 2001 o després (%)"]/100) * (TO_YEAR-2000))

indicadors_habitatge = indicadors_habitatge[["any", "CUSEC", "Edat_mitjana"]]

<ipython-input-52-1515a50f909f>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indicadors_habitatge['CUSEC'] = indicadors_habitatge['CUSEC'].str[:5] + indicadors_habitatge['CUSEC'].str[6:]


In [ ]:
for year in range(FROM_YEAR, TO_YEAR+1):
    if year not in indicadors_habitatge["any"].unique():
        print("missing",year,"-> copying from",indicadors_habitatge["any"].max())
        last_year = indicadors_habitatge["any"].max()
        last_year_data = indicadors_habitatge[indicadors_habitatge['any'] == last_year].copy()
        last_year_data.loc[:,"any"] = last_year_data.loc[:,"any"].apply(lambda x: year)
        indicadors_habitatge = pd.concat([indicadors_habitatge, last_year_data], ignore_index=True)

missing 2018 -> copying from 2021
missing 2019 -> copying from 2021
missing 2020 -> copying from 2021
missing 2022 -> copying from 2021


### Integrar els dos datasets en un

In [ ]:
result_df = pd.merge(indicadors_habitatge, edat_mitjana, on=['CUSEC', 'any'], how='left')
result_df["Edat_mitjana"] = result_df.apply(lambda row: row["Edat_mitjana_y"] if pd.notna(row["Edat_mitjana_y"]) else row["Edat_mitjana_x"], axis=1)
result_df = result_df.drop(columns=["Edat_mitjana_x", "Edat_mitjana_y"])

In [ ]:
result_df[result_df["CUSEC"]=="0801901001"]

,any,CUSEC,Edat_mitjana
0,2021,0801901001,98.4
1390,2018,0801901001,93.8
2780,2019,0801901001,96.6
4170,2020,0801901001,97.9
5560,2022,0801901001,99.4


# **Export**

Exporting the dataset into /clean_data/ path

In [ ]:
CLEAN_DATA_PATH = ROOT + "/cleaning/clean_data/"
EDAT_MITJANA_OUTPUT_FILE = "edat_mitjana_clean.csv"

result_df.to_csv(CLEAN_DATA_PATH + EDAT_MITJANA_OUTPUT_FILE, index=False)